# Pipeline As A Notebook

## Download Required Files

Download the required files and place them in th configured location below.

#### Hud Zip to Tract Crosswalk Dataset  
2020 and 2022 crosswalk files are also included in the repo.  
URL:  
Expected Location: `{download_directory}/crosswalk/zip_tract/`

#### SVI Dataset
URL:  
Expected Location: `{download_directory}/svi/`

#### Zillow HVI Dataset
URL:  
Expected Location: `{download_directory}/zillow`


In [6]:
download_directory = "../chosen_data"
year = 2020

## Database Connection

In [7]:
import duckdb

# create a connection to a file called 'test_database.db'
con = duckdb.connect("../data/database_jupyter.db")

## Create Tables
If they do not exist create the expected tables

Would need to include code for table migration/translation when new SVI datasets are downloaded

We will not precreate Zillows Bronze table, as those will be overwrites exclusively and then need to be pivoted. 

In [8]:
con.sql("SHOW TABLES")

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘

In [9]:
# create_table_hud_cbsa_zip = "CREATE OR REPLACE TABLE crosswalk_cbsa_zip (CBSA BIGINT, ZIP BIGINT, RES_RATIO DOUBLE, BUS_RATIO DOUBLE, OTH_RATIO DOUBLE, TOT_RATIO DOUBLE, start_date DATE, end_date DATE, load_quarter VARCHAR)"
create_table_zillow_zillow_cbsa = "CREATE OR REPLACE TABLE crosswalk_zillow_cbsa (CountyName varchar, StateName varchar, StateFIPS INT64, CountyFIPS INT64, MetroName_Zillow varchar, CBSAName varchar, CountyRegionID_Zillow INT64, MetroRegionID_Zillow DOUBLE, FIPS INT64, CBSACode DOUBLE)"
# con.sql(create_table_hud_cbsa_zip)
con.sql(create_table_zillow_zillow_cbsa)

create_table_hud_zip_tract = "CREATE OR REPLACE TABLE crosswalk_zip_tract (ZIP BIGINT, TRACT BIGINT, RES_RATIO DOUBLE, BUS_RATIO DOUBLE, OTH_RATIO DOUBLE, TOT_RATIO DOUBLE, start_date DATE, end_date DATE, load_quarter VARCHAR)"
create_table_svi = "CREATE OR REPLACE TABLE svi (STATE VARCHAR, ST_ABBR VARCHAR, STCNTY BIGINT, COUNTY VARCHAR, FIPS BIGINT, LOCATION VARCHAR, RPL_THEME1 DOUBLE, RPL_THEME2 DOUBLE, RPL_THEME3 DOUBLE, RPL_THEME4 DOUBLE, RPL_THEMES DOUBLE, year BIGINT)"
con.sql(create_table_hud_zip_tract)
con.sql(create_table_svi)
con.sql("SHOW TABLES")

┌───────────────────────┐
│         name          │
│        varchar        │
├───────────────────────┤
│ crosswalk_zillow_cbsa │
│ crosswalk_zip_tract   │
│ svi                   │
└───────────────────────┘

## Bronze

### Cross Walk Tables

In [10]:
import glob
import pandas as pd
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

## Reads from disk in the expected format <type>_<end_month><year>.xlsx
files_to_load = glob.glob(f"{download_directory}/crosswalk/zip_tract/*{year}.xlsx")
pattern = r"(\w+)_(\w+)_(\d{2})(\d{4})\.xlsx"

dfs = []
for file in files_to_load:
    match = re.search(pattern, file)
    cola = match.group(1)
    colb = match.group(2)
    month = match.group(3)
    date = datetime.strptime(f"{year}-{month}-01", "%Y-%m-%d")
    start_date = date - relativedelta(months=2)

    if month == "12":
        end_date = date.replace(month=12, day=31)
    else:
        end_date = date.replace(month=date.month + 1, day=1) - timedelta(days=1)

    load_quarter = f"{month}{year}"

    # Load the XLSX, apply the start and end dates. Union, and Return
    df = pd.read_excel(file)[
        [cola.upper(), colb.upper(), "RES_RATIO", "BUS_RATIO", "OTH_RATIO", "TOT_RATIO"]
    ]
    df["start_date"] = start_date.strftime("%Y-%m-%d")
    df["end_date"] = end_date.strftime("%Y-%m-%d")
    df["load_quarter"] = load_quarter
    dfs += [df]
zip_tract_df = pd.concat(dfs, axis=0, ignore_index=True)

Delete and Insert into tables since these would be overwrite of entire years if loaded

In [11]:
con.sql(
    "DELETE FROM crosswalk_zip_tract WHERE load_quarter IN (SELECT DISTINCT load_quarter FROM zip_tract_df)"
)

In [12]:
con.sql(
    """
INSERT INTO crosswalk_zip_tract FROM
    (SELECT * FROM zip_tract_df);
           """
)

In [13]:
con.sql("SELECT DISTINCT load_quarter FROM crosswalk_zip_tract")

┌──────────────┐
│ load_quarter │
│   varchar    │
├──────────────┤
│ 032020       │
│ 092020       │
│ 122020       │
│ 062020       │
└──────────────┘

### SVI

In [14]:
import glob
import pandas as pd

# Find all crosswalk files in the directory
files_to_load = glob.glob(f"../chosen_data/svi/SVI_{year}_US.csv")

if not files_to_load:
    raise ValueError(f"No file found for year {year} in ../chosen_data/svi/!")
if len(files_to_load) > 1:
    raise ValueError(
        f"Multiple files found for {year} in ../chosen_data/svi/!\n{files_to_load}"
    )

file = files_to_load[0]
df_svi = pd.read_csv(file)[
    [
        "STATE",
        "ST_ABBR",
        "STCNTY",
        "COUNTY",
        "FIPS",
        "LOCATION",
        "RPL_THEME1",
        "RPL_THEME2",
        "RPL_THEME3",
        "RPL_THEME4",
        "RPL_THEMES",
    ]
]
df_svi["year"] = year

In [15]:
con.sql("DELETE FROM svi WHERE year IN (SELECT DISTINCT year FROM df_svi)")

con.sql(
    """
INSERT INTO svi FROM
    (SELECT * FROM df_svi);
           """
)

In [16]:
con.sql("SELECT * FROM svi")

┌────────────┬─────────┬────────┬────────────┬────────────┬───────────────────────────────────────────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬───────┐
│   STATE    │ ST_ABBR │ STCNTY │   COUNTY   │    FIPS    │                     LOCATION                      │ RPL_THEME1 │ RPL_THEME2 │ RPL_THEME3 │ RPL_THEME4 │ RPL_THEMES │ year  │
│  varchar   │ varchar │ int64  │  varchar   │   int64    │                      varchar                      │   double   │   double   │   double   │   double   │   double   │ int64 │
├────────────┼─────────┼────────┼────────────┼────────────┼───────────────────────────────────────────────────┼────────────┼────────────┼────────────┼────────────┼────────────┼───────┤
│ Alabama    │ AL      │   1001 │ Autauga    │ 1001020100 │ Census Tract 201, Autauga County, Alabama         │     0.4578 │     0.5079 │     0.3921 │     0.0945 │     0.2823 │  2020 │
│ Alabama    │ AL      │   1001 │ Autauga    │ 1001020200 │ Census Tract 20

### Zillow Datasets

In [17]:
import pandas as pd

df_zillow_zhvi_sfr_zip = pd.read_csv(
    "../chosen_data/zillow/Zip_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_month.csv"
)

In [18]:
con.sql(
    """
CREATE OR REPLACE TABLE zillow_zhvi_sfr_zip_bronze AS
    SELECT * FROM df_zillow_zhvi_sfr_zip;
           """
)

In [19]:
con.sql("SHOW TABLES")

┌────────────────────────────┐
│            name            │
│          varchar           │
├────────────────────────────┤
│ crosswalk_zillow_cbsa      │
│ crosswalk_zip_tract        │
│ svi                        │
│ zillow_zhvi_sfr_zip_bronze │
└────────────────────────────┘

In [20]:
con.sql("SELECT * FROM crosswalk_zip_tract")

┌───────┬─────────────┬────────────────────────┬───────────────────────┬──────────────────────┬───────────────────────┬────────────┬────────────┬──────────────┐
│  ZIP  │    TRACT    │       RES_RATIO        │       BUS_RATIO       │      OTH_RATIO       │       TOT_RATIO       │ start_date │  end_date  │ load_quarter │
│ int64 │    int64    │         double         │        double         │        double        │        double         │    date    │    date    │   varchar    │
├───────┼─────────────┼────────────────────────┼───────────────────────┼──────────────────────┼───────────────────────┼────────────┼────────────┼──────────────┤
│   501 │ 36103158607 │                    0.0 │                   1.0 │                  0.0 │                   1.0 │ 2020-01-01 │ 2020-03-31 │ 032020       │
│   601 │ 72001956800 │    0.01481610597873453 │   0.01256281407035176 │   0.0440251572327044 │  0.015411503018748014 │ 2020-01-01 │ 2020-03-31 │ 032020       │
│   601 │ 72113071700 │    0.16105

## Silver Tables

* crosswalk zillowid to tract
* Zillow pivoted
* Zillow with SVI


### Crosswalk Tracts per Zip

In [22]:
con.sql(
    """
CREATE OR REPLACE TABLE tracts_per_zip_silver AS
(      SELECT 
        ZIP as zip, 
        start_date, 
        end_date, 
        len(list(TRACT)) as len_tracts, 
        list(TRACT) as tracts 
      FROM (
          SELECT 
              ZIP, 
              TRACT, 
              start_date, 
              end_date 
          FROM crosswalk_zip_tract 
          GROUP BY 
              ZIP, 
              TRACT, 
              start_date, 
              end_date
          ) 
      GROUP BY
          ZIP, 
          start_date, 
          end_date 
      ORDER BY zip, start_date DESC)
"""
)

In [23]:
con.sql("SELECT * FROM tracts_per_zip_silver")

┌───────┬────────────┬────────────┬────────────┬──────────────────────────────────────────────────────┐
│  zip  │ start_date │  end_date  │ len_tracts │                        tracts                        │
│ int64 │    date    │    date    │   int64    │                       int64[]                        │
├───────┼────────────┼────────────┼────────────┼──────────────────────────────────────────────────────┤
│   501 │ 2020-10-01 │ 2020-12-31 │          1 │ [36103158607]                                        │
│   501 │ 2020-07-01 │ 2020-09-30 │          1 │ [36103158607]                                        │
│   501 │ 2020-04-01 │ 2020-06-30 │          1 │ [36103158607]                                        │
│   501 │ 2020-01-01 │ 2020-03-31 │          1 │ [36103158607]                                        │
│   601 │ 2020-10-01 │ 2020-12-31 │          4 │ [72001956700, 72113071700, 72001956800, 72001956600] │
│   601 │ 2020-07-01 │ 2020-09-30 │          4 │ [72001956700, 7

### Unpivoted Zillow

In [24]:
con.sql(
    """
CREATE OR REPLACE TABLE zillow_zhvi_sfr_zip_silver AS
    (UNPIVOT zillow_zhvi_sfr_zip_bronze
    ON COLUMNS(* EXCLUDE (RegionID, SizeRank, RegionName, RegionType, StateName, State, City, Metro, CountyName))
    INTO
    NAME date
    VALUE zhvi)
"""
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [25]:
con.sql("SELECT * FROM zillow_zhvi_sfr_zip_silver")

┌──────────┬──────────┬────────────┬────────────┬───────────┬─────────┬────────────┬──────────────────────────────────────┬────────────────────┬────────────┬────────────────────┐
│ RegionID │ SizeRank │ RegionName │ RegionType │ StateName │  State  │    City    │                Metro                 │     CountyName     │    date    │        zhvi        │
│  int64   │  int64   │   int64    │  varchar   │  varchar  │ varchar │  varchar   │               varchar                │      varchar       │  varchar   │       double       │
├──────────┼──────────┼────────────┼────────────┼───────────┼─────────┼────────────┼──────────────────────────────────────┼────────────────────┼────────────┼────────────────────┤
│    91982 │        1 │      77494 │ zip        │ TX        │ TX      │ Katy       │ Houston-The Woodlands-Sugar Land, TX │ Fort Bend County   │ 2000-01-31 │ 209467.46053724916 │
│    91982 │        1 │      77494 │ zip        │ TX        │ TX      │ Katy       │ Houston-The Woodland

### Zillow with Tracts

In [27]:
con.sql(
    """
CREATE OR REPLACE TABLE zillow_zhvi_sfr_zip_with_tracts_silver AS (
      SELECT 
          RegionName, 
          StateName,
          zip, 
          date, 
          zhvi, 
          list_sort(tracts) as tracts
      FROM (
          SELECT 
              * 
          FROM zillow_zhvi_sfr_zip_silver z 
          LEFT JOIN tracts_per_zip_silver cw 
          ON 
              z.RegionName = cw.zip AND 
              z.date::DATE >= cw.start_date AND 
              z.date::DATE <= cw.end_date
      ) WHERE tracts is not null)
"""
)

In [28]:
con.sql("SELECT * FROM zillow_zhvi_sfr_zip_with_tracts_silver WHERE RegionName = 21771")

┌────────────┬───────────┬───────┬────────────┬────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ RegionName │ StateName │  zip  │    date    │        zhvi        │                                                                                  tracts                                                                                   │
│   int64    │  varchar  │ int64 │  varchar   │       double       │                                                                                  int64[]                                                                                  │
├────────────┼───────────┼───────┼────────────┼────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│      21771 │ MD        │ 21771 │ 2

### Zillow HVI with SVI

In [30]:
con.sql(
    """
CREATE OR REPLACE TABLE zillow_zhvi_sfr_zip_with_svi_silver AS (
          WITH filtered_svi AS (SELECT 
              RegionName,
              StateName,
              zip,
              date,
              zhvi,
              list_filter(list(RPL_THEME1), x -> x > -999) AS available_rpl_theme1,
              list_filter(list(RPL_THEME2), x -> x > -999) AS available_rpl_theme2,
              list_filter(list(RPL_THEME3), x -> x > -999) AS available_rpl_theme3,
              list_filter(list(RPL_THEME4), x -> x > -999) AS available_rpl_theme4,
              list_filter(list(RPL_THEMES), x -> x > -999) AS available_rpl_themes,
            FROM (SELECT * FROM zillow_zhvi_sfr_zip_with_tracts_silver z LEFT JOIN svi s ON s.year = year(z.date::DATE) AND s.FIPS IN z.tracts) GROUP BY         
              RegionName,
              StateName,
              zip,
              date,
              zhvi)
      SELECT  RegionName,
              StateName,
              zip,
              date,
              zhvi,
              list_aggregate(available_rpl_theme1, 'mean') as avg_rpl_theme1,
              list_aggregate(available_rpl_theme2, 'mean') as avg_rpl_theme2,
              list_aggregate(available_rpl_theme3, 'mean') as avg_rpl_theme3,
              list_aggregate(available_rpl_theme4, 'mean') as avg_rpl_theme4,
              list_aggregate(available_rpl_themes, 'mean') as avg_rpl_themes
              FROM filtered_svi 
)
"""
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [31]:
con.sql("SELECT * FROM zillow_zhvi_sfr_zip_with_svi_silver ORDER BY zip, date").show(
    max_rows=100
)

┌────────────┬───────────┬───────┬────────────┬────────────────────┬──────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬──────────────────────┐
│ RegionName │ StateName │  zip  │    date    │        zhvi        │    avg_rpl_theme1    │   avg_rpl_theme2    │   avg_rpl_theme3    │   avg_rpl_theme4    │    avg_rpl_themes    │
│   int64    │  varchar  │ int64 │  varchar   │       double       │        double        │       double        │       double        │       double        │        double        │
├────────────┼───────────┼───────┼────────────┼────────────────────┼──────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┼──────────────────────┤
│       1001 │ MA        │  1001 │ 2020-01-31 │ 241850.05017537635 │               0.2487 │  0.5203166666666666 │ 0.21806666666666666 │              0.4713 │   0.3374333333333333 │
│       1001 │ MA        │  1001 │ 2020-02-29 │ 242785.81596241705 │               0.2487 │  0.

In [32]:
con.commit()
con.close()